In [116]:
import re
import pandas as pd
import chardet


In [117]:
def update_language(texto):
    replacements = {
        # Actualización del idioma
        # Actualiza de la letra 'c' a la 'k' si no está seguida de una 'h'
        r'c(?![h])': 'k',
        # Actualiza de la letra 'qu' a la 'k'
        r'qu': 'k',
        r'Qu': 'K',
        # Actualiza de la palabra 's̈h' por 'x' o
        r's̈h': 'x',
        r'S̈H': 'X',
        # Actualiza de la palabra 'hu[a|e|i]' por 'w[a|e|i]'
        r'hu(?=[e|a|i|é|á|í])': 'w',
        r'Hu(?=[e|a|i|é|á|í])': 'W',
        r'ë': 'e',
        r'Ë': 'E',
        r'ʆ': 'x',
        # r'ã': 'á',
    }

    for pattern, replacement in replacements.items():
        texto = re.sub(pattern, replacement, texto)

    return texto




In [118]:
df2 = pd.read_csv('CSV/U20114818_corpus.csv', sep='|', encoding='Windows-1252')
df2.to_csv('CSV/U20114818_corpus_utf-8.csv', index=False, encoding='utf-8', sep=',')

In [119]:
df1 = pd.read_csv('CSV/U20172563_corpus.csv', sep=',')
df2 = pd.read_csv('CSV/U20114818_corpus_utf-8.csv', sep=',')
df3 = pd.read_csv('CSV/U20190876_corpus.csv', sep=';')
df4 = pd.read_csv('CSV/U20195539_corpus_utf-8.csv', sep=',')
df5 = pd.read_csv('CSV/U20190390_corpus.csv', sep=';')

# Unir los dataframes
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
len(df)

148

In [120]:
def preprocess_text(df, column):
    # El body no debe tener cambios de línea
    df[column] = df[column].str.replace('\n', ' ')

    # Aplicar update_language a la columna Body
    df[column] = df[column].apply(update_language)

    # Todo el texto en minúsculas
    df[column] = df[column].str.lower()

    # Eliminar guiones
    df[column] = df[column].str.replace('-', '')
    df[column] = df[column].str.replace('—', '')

    # Eliminar espacios al inicio y al final
    df[column] = df[column].str.strip()

    # Todas las vocales sin tilde

    # Diccionarios para el mapeo de caracteres
    map_upper = {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    map_lower = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'}

    # Aplicar los mapeos
    for original, replacement in map_upper.items():
        df[column] = df[column].str.replace(original, replacement)

    for original, replacement in map_lower.items():
        df[column] = df[column].str.replace(original, replacement)

    return df

In [121]:
df = preprocess_text(df, 'Body')
df = preprocess_text(df, 'Title')

In [122]:
# Crear un vocabulario con las palabras de la columna Body
vocabulary = {}

def add_words_to_vocabulary(dataframe):
    for i, row in dataframe.iterrows():
        body = row['Body']
        # Eliminar las comillas triples al inicio y al final del cuerpo
        body = body[1:-1]
        # Separar el cuerpo en palabras
        palabras = set(body.split())  # use a set to remove duplicates
        for palabra in palabras:
            # Eliminar los signos de puntuación
            palabra = re.sub(r'[^\w\s]', '', palabra)
            # Convertir la palabra a minúsculas
            palabra = palabra.lower()
            if palabra in vocabulary:
                vocabulary[palabra] += 1
            else:
                vocabulary[palabra] = 1


In [123]:
add_words_to_vocabulary(df)

In [124]:
# Ordenar el diccionario por frecuencia de palabras
sorted_vocabulary = dict(sorted(vocabulary.items(), key=lambda item: item[1], reverse=True))
sorted_vocabulary.__len__()

7778

In [125]:
# Obtener las palabras con solo una ocurrencia
palabras_unicas = {k: v for k, v in sorted_vocabulary.items() if v == 1}
palabras_unicas.__len__()

5695

In [126]:
# Identificar en que cuentos se encuentran las palabras únicas
def find_unique_words(dataframe):
    unique_words = []
    for i, row in dataframe.iterrows():
        body = row['Body']
        # Eliminar las comillas triples al inicio y al final del cuerpo
        body = body[1:-1]
        # Separar el cuerpo en palabras
        palabras = set(body.split())  # use a set to remove duplicates
        for palabra in palabras:
            # Eliminar los signos de puntuación
            palabra = re.sub(r'[^\w\s]', '', palabra)
            # Convertir la palabra a minúsculas
            palabra = palabra.lower()
            if palabra in palabras_unicas:
                unique_words.append((row['Year'], row['Title'], palabra))
    return unique_words

unique_words = find_unique_words(df)

In [127]:
## Reeplace fake words

def replace_fake_words(texto):
    replacements = {
        r'aqkin': 'akin',
        r'kochi': 'koshi',
    }

    for pattern, replacement in replacements.items():
        texto = re.sub(pattern, replacement, texto)

    return texto


In [128]:
# mostrar tamaño de unique_words antes de eliminar fake words
print(len(unique_words))
# Aplicar replace_fake_words a la columna Body
df['Body'] = df['Body'].apply(replace_fake_words)
unique_words = find_unique_words(df)
print(len(unique_words))

5695
5693


In [129]:
# Este es el alfabeto shipibo-konibo: < a, b, ch, e, i, j, k, m, n, o, p, r, s, sh, x, t, ts, w, y>
# Las vocales son a, e, i, o
# Las vocales pueden llevar tilde: á, é, í, ó
# Identificar palabras que no pertenecen al alfabeto shipibo-konibo
def find_non_shipibo_words(dataframe):
    non_shipibo_words = []
    for i, row in dataframe.iterrows():
        body = row['Body']
        body = body[1:-1]
        palabras = set(body.split())
        for palabra in palabras:
            palabra = re.sub(r'[^\w\s]', '', palabra)
            # palabra = palabra.lower()
            if not re.match(r'^[a-záéíó]*(b|ch|e|i|j|k|m|n|o|p|r|s|sh|x|t|ts|w|y|á|é|í|ó)*$', palabra):
                non_shipibo_words.append((row['Year'], row['Title'], palabra))
    return non_shipibo_words

non_shipibo_words = find_non_shipibo_words(df)
print(len(non_shipibo_words))

39


In [130]:
non_shipibo_words

[(1979, 'ino betan yawish ini', 'señor'),
 (1973, 'cabayo betan shinobo ini', 'kabeyo_en'),
 (2005, 'jaen noa ika mayai', '5'),
 (2005, 'jaen noa ika mayai', '2816'),
 (2005, 'jaen noa ika mayai', '9'),
 (2005, 'jaen noa ika mayai', '194'),
 (2005, 'jaen noa ika mayai', '506'),
 (2005, 'jaen noa ika mayai', '154'),
 (2005, 'jaen noa ika mayai', '247'),
 (2005, 'jaen noa ika mayai', '1993'),
 (2005, 'jaskatash limonjema pikota', '32'),
 (2019, 'maxo naikan kani', 'cañiwa'),
 (2019, 'maxo naikan kani', 'kañiwa'),
 (2015, 'anishaman joa ikaxbi basimabires otsiax mawatai', '40'),
 (2015, 'anishaman joa ikaxbi basimabires otsiax mawatai', '70'),
 (2015, 'anishaman joa ikaxbi basimabires otsiax mawatai', '3'),
 (2023, 'kena jaweki irake aka i', 'b5'),
 (2023, 'kena jaweki irake aka i', 'a17'),
 (2023, 'kena jaweki irake aka i', '6maotianra'),
 (2023, 'kena jaweki irake aka i', 'a17'),
 (2023, 'kena jaweki irake aka i', '1'),
 (2023, 'jain pacho maw kanai karretera', '6'),
 (2023, 'peokin yoi

In [131]:
# Eliminar las letras que no pertenecen al alfabeto shipibo-konibo a excepción de números y la letra 'ñ'
def remove_non_shipibo_letters(texto):
    replacements = {
        r'[^a-zA-Záéíóñ0-9,\.;\-\'\s]': '',
    }

    for pattern, replacement in replacements.items():
        texto = re.sub(pattern, replacement, texto)

    return texto


# non_shipibo_words_before = find_non_shipibo_words(df)
# print(len(non_shipibo_words))
# Aplicar remove_non_shipibo_letters a la columna Body
df['Body'] = df['Body'].apply(remove_non_shipibo_letters)
#  Eliminar espacios extra dentro de los strings
df['Body'] = df['Body'].str.replace(' +', ' ', regex=True)


In [132]:
# Guardar el dataframe en un archivo CSV con las columnas Year, Title, Body y Category
df.to_csv('CSV/corpus.csv', index=False)

In [133]:
# non_shipibo_words = find_non_shipibo_words(df)
# print(len(non_shipibo_words))
# non_shipibo_words

# # words which have been removed
# removed_words = list(set(non_shipibo_words_before) - set(non_shipibo_words))
# removed_words


In [134]:
# Guardar el diccionario en un archivo CSV
# with open('CSV/vocabulary.csv', 'w', encoding='utf-8') as archivo:
#     archivo.write('Word,Frequency\n')
#     for palabra, frecuencia in sorted_vocabulary.items():
#         archivo.write(f'{palabra},{frecuencia}\n')